# X_score 도출
### 피어슨 상관관계 정도를 기준으로 수요 지수 추출
x_score = x1 * w1/(w1+w2)  +   x2 * w2/(w1+w2)

In [2]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

data = gpd.read_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인시_Grid_filtered_v6(gas_dist).geojson')

# 데이터 변환 
mMscaler = MinMaxScaler()
mMscaler.fit(data['val'].values.reshape(-1,1))
mMscaled_data = mMscaler.transform(data['val'].values.reshape(-1,1))
data['mmscaled_val']=mMscaled_data

# 데이터 변환 
mMscaler = MinMaxScaler()
mMscaler.fit(data['ev_cnt'].values.reshape(-1,1))
mMscaled_data = mMscaler.transform(data['ev_cnt'].values.reshape(-1,1))
data['mmscaled_ev_cnt']=mMscaled_data

# 데이터 변환 
mMscaler = MinMaxScaler()
mMscaler.fit(data['gas_dist'].values.reshape(-1,1))
mMscaled_data = mMscaler.transform(data['gas_dist'].values.reshape(-1,1))
data['mmscaled_gas_dist']=mMscaled_data

# correlation 분석
df = data[['mmscaled_val','mmscaled_ev_cnt','mmscaled_gas_dist','res_cnt']]
df = df.corr(method='pearson')
# 주유소 정보는 상관성이 낮으므롤 drop
w_val, w_ev=df['res_cnt'][0],df['res_cnt'][1]

# Correlation 값 기준 가중치 곱 sum 한 결과를 x_score로 사용
tqdm.pandas()
data['x_score']= data.progress_apply(lambda row : row.mmscaled_val * w_val/(w_val+w_ev) + row.mmscaled_ev_cnt * w_ev/(w_val+w_ev),axis=1)

100%|██████████| 15935/15935 [00:00<00:00, 29099.87it/s]


In [3]:
data.to_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인시_Grid_filtered_v7(x_score).geojson',encoding='utf-8',index=False)